In [7]:
import pandas as pd

In [8]:
colunas_gerenicas = [
 # Localização
    'SG_UF_PROVA',
    # Notas
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO',

    # Questões socioeconômicas
    'Q001', 
    'Q002', 
    'Q005', 
    'Q006', 
    'Q025',
    'TP_FAIXA_SALARIAL',

     # Características do candidato
    'TP_SEXO',
    'TP_COR_RACA',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_ST_CONCLUSAO',
    'TP_FAIXA_ETARIA',
]

colunas_geral = [
    # Presença
    'TP_PRESENCA_CN', 
    'TP_PRESENCA_CH', 
    'TP_PRESENCA_LC', 
    'TP_PRESENCA_MT',
    'TP_PRESENCA_GERAL',
    'TP_PRESENCA_REDACAO',
]

colunas_aspectos_sociais = [
    # Variáveis sociais
    'TP_ESTADO_CIVIL',
    'TP_LOCALIZACAO_ESC',

    # Infraestrutura
    'NU_INFRAESTRUTURA',
]

colunas_desempenho = [
    # Categorias de desempenho
    'NU_DESEMPENHO',
]

In [9]:
# Carregar microdados completos
arquivo = '../Iniciação Científica/Códigos/microdados_tratado.parquet'

# Especificar explicitamente o engine
microdados = pd.read_parquet(arquivo, engine='pyarrow')

microdados = microdados.sample(1500000, random_state=42)

# Adicionar os dtypes pre definidos - usando try/except para evitar erros
try:
    dtypes = pd.read_json("dtypes.json", typ='series')
    microdados = microdados.astype(dtypes.to_dict())
except Exception as e:
    print(f"Aviso ao aplicar dtypes: {e}")
    # Continuar com os tipos atuais

# Criar arquivos separados para cada aba
microdados[colunas_gerenicas].to_parquet('sample_gerenico.parquet', index=False, engine='pyarrow')
microdados[colunas_geral].to_parquet('sample_geral.parquet', index=False, engine='pyarrow')
microdados[colunas_aspectos_sociais].to_parquet('sample_aspectos_sociais.parquet', index=False, engine='pyarrow')
microdados[colunas_desempenho].to_parquet('sample_desempenho.parquet', index=False, engine='pyarrow')

# Criar arquivos dtypes separados
dtypes_gerenico = {col: str(microdados[col].dtype) for col in colunas_gerenicas if col in microdados.columns}
dtypes_geral = {col: str(microdados[col].dtype) for col in colunas_geral if col in microdados.columns}
dtypes_aspectos = {col: str(microdados[col].dtype) for col in colunas_aspectos_sociais if col in microdados.columns}
dtypes_desempenho = {col: str(microdados[col].dtype) for col in colunas_desempenho if col in microdados.columns}

# Usar json module em vez do pandas to_json para evitar problemas de serialização
import json
with open('dtypes_gerenico.json', 'w') as f:
    json.dump(dtypes_gerenico, f)
with open('dtypes_geral.json', 'w') as f:
    json.dump(dtypes_geral, f)
with open('dtypes_aspectos_sociais.json', 'w') as f:
    json.dump(dtypes_aspectos, f)
with open('dtypes_desempenho.json', 'w') as f:
    json.dump(dtypes_desempenho, f)

In [10]:
# arquivo = '../Iniciação Científica/Códigos/microdados_tratado.parquet'

# microdados = pd.read_parquet(arquivo, engine='pyarrow',)

# dtypes = pd.read_json("dtypes.json", typ='series')
# microdados = microdados.astype(dtypes)

# microdados = microdados[colunas_desejadas]

# microdados = microdados[(microdados['TP_PRESENCA_GERAL'] == 1) & (microdados['NU_MEDIA_GERAL'] != -1)]

# microdados = microdados[microdados['NU_MEDIA_GERAL'] != -1]

# microdados = microdados.sample(938488, random_state=42)

# microdados.to_parquet('sample.parquet', index=False, engine='pyarrow')

In [11]:
# df = pd.read_parquet('sample.parquet')
# f = df.select_dtypes(include='float16').columns
# i = df.select_dtypes(include='int16').columns

# df[f] = df[f].astype('float64')
# df[i] = df[i].astype('int64')
# profile = ProfileReport(df, title="Analise microdados")
# profile.to_file("exploratoria_original.html")